<a href="https://colab.research.google.com/github/AG10009/openstates-scrapers/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files


uploaded = files.upload()


Saving crime_data_by_state.xlsx to crime_data_by_state.xlsx


In [3]:
import pandas as pd
import io

ori_data = pd.read_excel(io.BytesIO(uploaded['crime_data_by_state.xlsx']))
print(ori_data)


             ori       counties  is_nibrs   latitude   longitude state_abbr  \
0      AL0430200            LEE      True  32.604064  -85.353048         AL   
1      AL0430100            LEE      True  32.608086  -85.475136         AL   
2      AL0430000            LEE      True  32.604064  -85.353048         AL   
3      AL0070100           BIBB      True  33.015893  -87.127148         AL   
4      AL0070000           BIBB      True  32.939995  -87.116830         AL   
...          ...            ...       ...        ...         ...        ...   
19161  WY0190000     SWEETWATER      True  41.660339 -108.875676         WY   
19162  WY0090000    HOT SPRINGS      True  43.647200 -108.208950         WY   
19163  WY0090100    HOT SPRINGS      True  43.720871 -108.435652         WY   
19164  WYWHP0000  NOT SPECIFIED      True  41.167576 -104.841750         WY   
19165  WYDI05000  NOT SPECIFIED      True  43.055303 -108.605531         WY   

      state_name                          agency_na

In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

api_key = 'TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG'

agency_url_template = 'https://api.usa.gov/crime/fbi/cde/arrest/agency/{}/all?type=counts&from=01-2023&to=12-2023&API_KEY={}'

def get_agency_data(ori, api_key):
    url = agency_url_template.format(ori, api_key)
    print(f"Querying URL: {url}")
    response = requests.get(url)

    if response.status_code == 200:
        try:
            data = response.json()
            return {
                'ori': ori,
                'rates': data.get('rates', {}),
                'actuals': data.get('actuals', {}),
                'tooltips': data.get('tooltips', {}),
                'populations': data.get('populations', {}),
                'participated_population': data.get('participated_population', {}),
                'cde_properties': data.get('cde_properties', {})
            }
        except ValueError:
            print(f"Invalid JSON response for ORI {ori}. Response: {response.text}")
            return None
    else:
        print(f"Failed to retrieve data for ORI {ori}: {response.status_code}")
        return None

all_agency_data = []

max_workers = 10

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(get_agency_data, ori, api_key): ori for ori in ori_data['ori']}

    for future in as_completed(futures):
        ori = futures[future]
        try:
            result = future.result()
            if result:
                all_agency_data.append(result)
        except Exception as exc:
            print(f'ORI {ori} generated an exception: {exc}')

df = pd.DataFrame(all_agency_data)

print(df.head())

df.to_csv('detailed_agency_crime_data.csv', index=False)

print("Data collection and storage complete.")


Querying URL: https://api.usa.gov/crime/fbi/cde/arrest/agency/AL0430200/all?type=counts&from=01-2023&to=12-2023&API_KEY=TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG
Querying URL: https://api.usa.gov/crime/fbi/cde/arrest/agency/AL0430100/all?type=counts&from=01-2023&to=12-2023&API_KEY=TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG
Querying URL: https://api.usa.gov/crime/fbi/cde/arrest/agency/AL0430000/all?type=counts&from=01-2023&to=12-2023&API_KEY=TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG
Querying URL: https://api.usa.gov/crime/fbi/cde/arrest/agency/AL0070100/all?type=counts&from=01-2023&to=12-2023&API_KEY=TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG
Querying URL: https://api.usa.gov/crime/fbi/cde/arrest/agency/AL0070000/all?type=counts&from=01-2023&to=12-2023&API_KEY=TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG
Querying URL: https://api.usa.gov/crime/fbi/cde/arrest/agency/AL0070300/all?type=counts&from=01-2023&to=12-2023&API_KEY=TSxmGFYJYOBGobxJda1ygsejrCaD98in2GCaP2TG
Querying URL: https://api.usa.gov/